# Covid prediction on X-ray images

## First let's split data into train/validation folders with images in 80/20 ratio

In [1]:
import os
import numpy as np
import shutil

# # Creating Train / Val / Test folders (One time use)
root_dir = 'X-ray'
positive_class = 'COVID'
negative_class = 'Non-COVID'
try:
    os.makedirs(root_dir +'/train/' + positive_class)
    os.makedirs(root_dir +'/train/' + negative_class)
    os.makedirs(root_dir +'/val/' + positive_class)
    os.makedirs(root_dir +'/val/' + negative_class)
    
    # Creating partitions of the data after shuffling
    for class_covid in [positive_class,negative_class]:
        currentCls = class_covid
        src = root_dir + "/" + currentCls # Folder to copy images from

        allFileNames = os.listdir(src)
        np.random.shuffle(allFileNames)
        train_FileNames, val_FileNames = np.split(np.array(allFileNames),[int(len(allFileNames)*0.8)])


        train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
        val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]

        print('Total images: ',currentCls, len(allFileNames))
        print('Training: ', len(train_FileNames))
        print('Validation: ', len(val_FileNames))

        # Copy-pasting images
        for name in train_FileNames:
            shutil.copy(name, root_dir+"/train/"+currentCls)

        for name in val_FileNames:
            shutil.copy(name, root_dir+"/val/"+currentCls)
except:
    pass

In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(42)

In [3]:
transformations = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
train_set = datasets.ImageFolder(root_dir+"/train", transform = transformations)
val_set = datasets.ImageFolder(root_dir+"/val", transform = transformations)

In [5]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size =32, shuffle=True)

### Setting up the model

In [6]:
mod = models.googlenet(pretrained=True)

In [7]:
num_labels = 2

# Dropout applied after pretrained model as not all features are directly useful

model = nn.Sequential(mod,
                      nn.Dropout(p=0.2),
                      nn.Linear(1000, 128),
                      nn.ReLU(),
                      nn.Linear(128, num_labels),
                      nn.LogSoftmax(dim=1)
                     )

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Sequential(
  (0): GoogLeNet(
    (conv1): BasicConv2d(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): BasicConv2d(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv3): BasicConv2d(
      (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (inception3a): Inception(
      (branch1): BasicConv2d(
        (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): Batc

### Phase 1: Feature extraction 

I.e. Training the additional layers we attach for our task

In [9]:
# Freeze pretrained layers

for param in mod.parameters():
    param.requires_grad = False

In [10]:
# Set the error function using torch.nn as nn library
criterion = nn.NLLLoss()
# Set the optimizer function using torch.optim as optim library
optimizer = optim.Adam(model.parameters(),weight_decay=0.0001)

In [11]:
epochs = 5
for epoch in range(epochs):
    train_loss = 0
    val_loss = 0
    accuracy = 0
    
    # Training the model
    model.train()
    counter = 0
    for inputs, labels in train_loader:
        # Move to device
        inputs, labels = inputs.to(device), labels.to(device)
        # Clear optimizers
        optimizer.zero_grad()
        # Forward pass
        output = model.forward(inputs)
        # Loss
        loss = criterion(output, labels)
        # Calculate gradients (backpropogation)
        loss.backward()
        # Adjust parameters based on gradients
        optimizer.step()
        # Add the loss to the training set's rnning loss
        train_loss += loss.item()*inputs.size(0)
        
        # Print the progress of our training
        counter += 1
#            print(counter, "/", len(train_loader))
        
    # Evaluating the model
    model.eval()
    counter = 0
    # Tell torch not to calculate gradients
    with torch.no_grad():
        for inputs, labels in val_loader:
            # Move to device
            inputs, labels = inputs.to(device), labels.to(device)
            # Forward pass
            output = model.forward(inputs)
            # Calculate Loss
            valloss = criterion(output, labels)
            # Add loss to the validation set's running loss
            val_loss += valloss.item()*inputs.size(0)
            
            # Since our model outputs a LogSoftmax, find the real 
            # percentages by reversing the log function
            output = torch.exp(output)
            # Get the top class of the output
            top_p, top_class = output.topk(1, dim=1)
            # See how many of the classes were correct?
            equals = top_class == labels.view(*top_class.shape)
            # Calculate the mean (get the accuracy for this batch)
            # and add it to the running accuracy for this epoch
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
            # Print the progress of our evaluation
            counter += 1

#                print(counter, "/", len(val_loader))
    
    # Get the average loss for the entire epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = val_loss/len(val_loader.dataset)
    # Print out the information
    print('Accuracy: ', accuracy/len(val_loader))
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss, valid_loss))

Accuracy:  0.9239004629629629
Epoch: 1 	Training Loss: 0.288212 	Validation Loss: 0.219270
Accuracy:  0.9403935185185185
Epoch: 2 	Training Loss: 0.228049 	Validation Loss: 0.176710
Accuracy:  0.9424189814814815
Epoch: 3 	Training Loss: 0.204476 	Validation Loss: 0.170155
Accuracy:  0.9412615740740741
Epoch: 4 	Training Loss: 0.192522 	Validation Loss: 0.161253
Accuracy:  0.9516782407407407
Epoch: 5 	Training Loss: 0.185239 	Validation Loss: 0.143293
Accuracy:  0.9482060185185185
Epoch: 6 	Training Loss: 0.171185 	Validation Loss: 0.145605
Accuracy:  0.9291087962962963
Epoch: 7 	Training Loss: 0.172259 	Validation Loss: 0.185210


### Phase 2: Fine-tuning the full model

In [12]:
## UNFREEZING THE PRETRAINED LAYERS

for param in mod.parameters():
    param.requires_grad = True
    
criterion = nn.NLLLoss()

# Lowered weight decay & learning rate for fine-tuning
optimizer = optim.Adam(model.parameters(),weight_decay=0.0001,lr=5e-5)

In [13]:
epochs = 5
for epoch in range(epochs):
    train_loss = 0
    val_loss = 0
    accuracy = 0
    
    # Training the model
    model.train()
    counter = 0
    for inputs, labels in train_loader:
        # Move to device
        inputs, labels = inputs.to(device), labels.to(device)
        # Clear optimizers
        optimizer.zero_grad()
        # Forward pass
        output = model.forward(inputs)
        # Loss
        loss = criterion(output, labels)
        # Calculate gradients (backpropogation)
        loss.backward()
        # Adjust parameters based on gradients
        optimizer.step()
        # Add the loss to the training set's rnning loss
        train_loss += loss.item()*inputs.size(0)
        
        # Print the progress of our training
        counter += 1
 
        #print(counter, "/", len(train_loader))
        
    # Evaluating the model
    model.eval()
    counter = 0
    # Tell torch not to calculate gradients
    with torch.no_grad():
        for inputs, labels in val_loader:
            # Move to device
            inputs, labels = inputs.to(device), labels.to(device)
            # Forward pass
            output = model.forward(inputs)
            # Calculate Loss
            valloss = criterion(output, labels)
            # Add loss to the validation set's running loss
            val_loss += valloss.item()*inputs.size(0)
            
            # Since our model outputs a LogSoftmax, find the real 
            # percentages by reversing the log function
            output = torch.exp(output)
            # Get the top class of the output
            top_p, top_class = output.topk(1, dim=1)
            # See how many of the classes were correct?
            equals = top_class == labels.view(*top_class.shape)
            # Calculate the mean (get the accuracy for this batch)
            # and add it to the running accuracy for this epoch
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
            # Print the progress of our evaluation
            counter += 1

            #print(counter, "/", len(val_loader))
    
    # Get the average loss for the entire epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = val_loss/len(val_loader.dataset)
    # Print out the information
    print('Accuracy: ', accuracy/len(val_loader))
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss, valid_loss))

Accuracy:  0.9733796296296297
Epoch: 1 	Training Loss: 0.142669 	Validation Loss: 0.079408
Accuracy:  0.9791666666666666
Epoch: 2 	Training Loss: 0.087191 	Validation Loss: 0.055673
Accuracy:  0.9823495370370371
Epoch: 3 	Training Loss: 0.062172 	Validation Loss: 0.043640
Accuracy:  0.9846643518518519
Epoch: 4 	Training Loss: 0.046984 	Validation Loss: 0.041089
Accuracy:  0.9730902777777778
Epoch: 5 	Training Loss: 0.039144 	Validation Loss: 0.048602
Accuracy:  0.9829282407407407
Epoch: 6 	Training Loss: 0.038357 	Validation Loss: 0.050669
Accuracy:  0.9858217592592593
Epoch: 7 	Training Loss: 0.034958 	Validation Loss: 0.041328


In [14]:
torch.save(model,'985 ')

Sequential(
  (0): GoogLeNet(
    (conv1): BasicConv2d(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): BasicConv2d(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv3): BasicConv2d(
      (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (inception3a): Inception(
      (branch1): BasicConv2d(
        (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): Batc